##### 爬取PTT的資料
1. 輸入「要爬的網址」、「要爬的頁數」、「檔案名稱」
2. 爬取PTT（作者, 看板, 標題, 日期, 內文, 推文數, 噓文數, 箭頭留言數, 連結, 留言資訊）
3. 處理原始資料後，分別存成兩個csv檔：文章資料、留言資料

In [ ]:
# 爬網頁的程式
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import json

# 設定headers
headers = {
    'cookie': 'over18=1',
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36'
}

# 爬取文章的詳細資訊
def get_article_content(article_url):
    response = requests.get(article_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    meta_values = soup.find_all('span', class_='article-meta-value')
    if not meta_values:
        return None

    author, board, title, date = (meta.text for meta in meta_values)
    content = soup.find(id="main-content").text
    content = content.split(date)[-1].split('※ 發信站: 批踢踢實業坊(ptt.cc)')[0].split('\n', 1)[-1].rsplit('\n', 1)[0]

    # 爬取留言
    comments = []
    push_tags = soup.find_all('div', class_='push')
    for push_tag in push_tags:
        push_type = push_tag.find('span', class_='push-tag').text.strip()
        push_user = push_tag.find('span', class_='push-userid').text.strip()
        push_content = push_tag.find('span', class_='push-content').text.strip()[2:]
        push_ipdatetime = push_tag.find('span', class_='push-ipdatetime').text.strip()
        
        comment = {
            'type': push_type.strip(),
            'user': push_user,
            'content': push_content,
            'ipdatetime': push_ipdatetime
        }
        
        comments.append(comment)

    return {
        'author': author,
        'board': board,
        'title': title,
        'date': date,
        'content': content,
        'comments': comments,
        'link': article_url
    }

# 所有文章連結
def get_articles_list(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    return ['https://www.ptt.cc' + link.a['href'] for link in soup.find_all('div', class_='title') if link.a]

# 主函式: 爬取多頁，存儲結果
def main(url, pages, csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf_8_sig') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['作者', '看板', '標題', '日期', '內文', '推文數', '噓文數', '箭頭留言數', '連結', '留言資訊'])
        
    for _ in range(pages):
        article_urls = get_articles_list(url)
        for article_url in article_urls:
            article_data = get_article_content(article_url)
            if article_data:
                # 初始化推文、噓文和箭頭留言的計數器
                pushes = 0
                boos = 0
                arrows = 0
                # 遍歷留言列表來計算推文、噓文和箭頭留言的數量
                for comment in article_data['comments']:
                    if comment['type'] == '推':
                        pushes += 1
                    elif comment['type'] == '噓':
                        boos += 1
                    elif comment['type'] == '→':
                        arrows += 1
                # 將留言資訊轉換成json
                comments_json = json.dumps(article_data['comments'], ensure_ascii=False)
                with open(csv_file_path, 'a', newline='', encoding='utf_8_sig') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([article_data['author'], article_data['board'], article_data['title'], article_data['date'], article_data['content'], pushes, boos, arrows, article_data['link'], comments_json])
            time.sleep(0.5)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        next_page = 'https://www.ptt.cc' + soup.find('a', string='‹ 上頁')['href']
        url = next_page


# 替換成要爬取的PTT板網址、頁數、要儲存的檔名
main('https://www.ptt.cc/bbs/Bank_Service/index.html', 20, 'ptt_bank_text.csv')


In [2]:
# 將抓取的資料轉換成df
def info(filename):
    all_info = pd.read_csv(filename)
    return all_info


all_info = info('ptt_bank_textk.csv')  # 這裡要輸入檔名

print(f'總共抓取了{all_info.shape[0]}筆資料。\n')
print(f'第一筆資料：\n{all_info.loc[0, :].to_frame().T}')

總共抓取了102筆資料。

第一筆資料：
                作者            看板                             標題  \
0  monkkeywom (キラ)  Bank_Service  [情報] 星展digibank首登最高領200＋新舊戶抽獎   

                         日期  \
0  Thu Feb  1 20:46:29 2024   

                                                  內文 推文數 噓文數 箭頭留言數  \
0  去年底就有看過這活動，好像有名額限制\n發現到3月都還有，之前沒領過的可以趕快登入看看\n一...   5   0     0   

                                                  連結  \
0  https://www.ptt.cc/bbs/Bank_Service/M.17067915...   

                                                留言資訊  
0  [{"type": "推", "user": "yamiyami", "content": ...  


In [3]:
# 將標題單獨存成一個column
def title_processing(all_info):
    all_info['category'] = all_info['標題'].apply(lambda x: x.split(']')[0])
    all_info['category'] = all_info['category'].apply(lambda x: x.replace('[', ''))
    all_info['標題'] = all_info['標題'].apply(lambda x: x.split(']')[1])  
    return all_info


all_info = title_processing(all_info)

print(f'處理後的標題：{all_info.head()}')

處理後的標題：                       作者            看板                          標題  \
0         monkkeywom (キラ)  Bank_Service    星展digibank首登最高領200＋新舊戶抽獎   
1       calmness26 (一顆蘋果)  Bank_Service   中信銀APP連結icashPay新戶最高贈100點   
2  jackloutter (Clarkson)  Bank_Service      月初＋農曆年前資料大塞車 郵局系統上午被塞爆   
3           eubrgbf (叢雨廚)  Bank_Service            華南能否線上設定「本人」約定帳號   
4             kedine (雀榕)  Bank_Service    永豐DAWHO龍年匯好運 換匯抽日幣、switc   

                         日期  \
0  Thu Feb  1 20:46:29 2024   
1  Fri Feb  2 15:55:45 2024   
2  Fri Feb  2 20:27:36 2024   
3  Fri Feb  2 21:03:50 2024   
4  Fri Feb  2 21:11:30 2024   

                                                  內文  推文數  噓文數  箭頭留言數  \
0  去年底就有看過這活動，好像有名額限制\n發現到3月都還有，之前沒領過的可以趕快登入看看\n一...    5    0      0   
1  中信Home Bank APP銀行存款帳戶連結icash Pay贈OPENPOINT點數\n...   10    0      5   
2  月初＋農曆年前資料大塞車 郵局系統上午被塞爆\nhttps://www.ctee.com.t...    1    1      2   
3  不好意思 需要大家求助\n小弟有購入一些美金 想換匯成台幣\n但App顯示沒有設約定帳號\n...    2    0     24   
4  活動內容：龍年匯好運活

In [4]:
# 將column轉換成英文，重新排序
def column_name_processing(dll_info):
    all_info = dll_info.rename(columns={ '作者': 'author','看板': 'board','標題': 'title','日期': 'date','內文': 'content',
                    '推文數': 'push','噓文數': 'boo','箭頭留言數': 'arrow','連結': 'link','留言資訊': 'comment_info'
                    })
    
    all_info = all_info[['board', 'author', 'category', 'title', 'date', 'content', 'push', 'boo', 'arrow', 'comment_info', 'link']]

    return all_info  


all_info = column_name_processing(all_info)

print(f'重新處理欄位後：\n{all_info.head(1)}')

重新處理欄位後：
          board           author category                      title  \
0  Bank_Service  monkkeywom (キラ)       情報   星展digibank首登最高領200＋新舊戶抽獎   

                       date  \
0  Thu Feb  1 20:46:29 2024   

                                             content  push  boo  arrow  \
0  去年底就有看過這活動，好像有名額限制\n發現到3月都還有，之前沒領過的可以趕快登入看看\n一...     5    0      0   

                                        comment_info  \
0  [{"type": "推", "user": "yamiyami", "content": ...   

                                                link  
0  https://www.ptt.cc/bbs/Bank_Service/M.17067915...  


In [9]:
# 將所有留言取出，另外存成df
def comment_processing(all_info):

    # 從所有的資料集取出留言，存成df
    all_comments = pd.DataFrame()
    for i in all_info.index:
        comments = json.loads(all_info.loc[i, 'comment_info'])
        comments_df = pd.DataFrame(comments)
        comments_df['source_index'] = i
        all_comments = pd.concat([all_comments, comments_df], ignore_index=True)

    # 重新命名留言的column，以免和原本的資料集搞混
    all_comments.rename(columns={'user': 'commenter', 'content': 'comment'}, inplace=True)

    # 將留言的ip和datetime分開
    all_comments['date_time'] = all_comments['ipdatetime'].apply(lambda x: x[-11:])
    all_comments['ip'] = all_comments['ipdatetime'].apply(lambda x: x[:-12])
    all_comments = all_comments.drop(['ipdatetime'], axis=1)

    return all_comments


all_comments = comment_processing(all_info)

print(f'{all_comments}')

     type   commenter                      comment  source_index    date_time  \
0       推    yamiyami                 有拿到首次app的100             0  02/02 01:29   
1       推      skye11          之前有領過了 還是推個～抽獎就碰碰運氣             0  02/02 13:40   
2       推  autumn0121                        有領過+1             0  02/02 14:22   
3       推     bigsara      原本看這活動好像到去年底而已，後來又延長到3月             0  02/02 17:42   
4       推  pippen2002            首次登入app?? 誰還0登入過?             0  02/02 20:51   
...   ...         ...                          ...           ...          ...   
3960    →      TZUYIC  想要盜刷也沒錢可以授權，0元的話連綁定授權都不會過，我           101  01/11 19:40   
3961    →      TZUYIC                       有試過。XD           101  01/11 19:40   
3962    →      TZUYIC  真的要提/轉或要用那張卡消費之前，再轉帳進來就可以了。           101  01/11 19:41   
3963    →    qqplusqq               這也是一個方式，避免被盜刷～           101  01/12 00:21   
3964    →  pippen2002                        推高手!!           101  01/14 20:33   

     ip  
0        
1      

In [10]:
# 將結果存成csv檔
def save_to_csv(all_info, all_comments):
    all_info.to_csv('all_info.csv', index=False, encoding='utf-8-sig')
    all_comments.to_csv('all_comments.csv', index=False, encoding='utf-8-sig')


save_to_csv(all_info, all_comments)